In [1]:
import sys
import os
from typing import Tuple
import pandas

sys.path.append('/home/danillorp/Área de Trabalho/github/fema/src/')


import fema_semi
import fema_classifier
import numpy as np
import pandas as pd
from sklearn.datasets import fetch_openml
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
import matplotlib.pyplot as plt
from sklearn import preprocessing

DEBUG = False

/tmp/ipykernel_101962/644247639.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas


In [2]:
ml_datasets = ['credit-g','airlines','satellite']

#'Titanic','vehicle','adult','SpeedDating','blood-transfusion-service-center','mnist_784','one-hundred-plants-texture','steel-plates-fault',
#'kr-vs-kp','arrhythmia','bank-marketing','PhishingWebsites','kc1','pc1','cmc','mfeat-factors','KDDCup09_appetency'
print('The experiment will run for ',len(ml_datasets))        

The experiment will run for  3


In [3]:
def experiment(dataset_name:str, data_x:pandas.core.frame.DataFrame, data_y:pandas.core.series.Series, test_size:float, n_runs:int, DEBUG:bool=False):
    for i in range(n_runs):
        #split datasets
        train_x, test_x, train_y, test_y = train_test_split(data_x, data_y, test_size=test_size)
        uknw_x, test_x, uknw_y, test_y = train_test_split(test_x, test_y, test_size=test_size)
    
        if DEBUG:
            print('**********',dataset_name,'**********')
            print('train_size',len(train_x),'classes:',set(train_y))
            print('test_size',len(test_x),'classes:',set(test_y))
            print('uknw_size',len(uknw_x),'classes:',set(uknw_y))
    
        #reorganize the arrays to be an input of FEMa library
        train_y = train_y[:,np.newaxis]
        test_y = test_y[:,np.newaxis]
        uknw_y = uknw_y[:,np.newaxis]
    
        #Apply data scaling based on training data
        scaler = StandardScaler()
    
        train_x = scaler.fit_transform(train_x)
        test_x = scaler.transform(test_x)
        uknw_x = scaler.transform(uknw_x)
    
        #Run the model without the SemiSupervised
        model_fema_original = fema_classifier.FEMaClassifier(k=50,basis=fema_classifier.Basis.shepardBasis)
        model_fema_original.fit(train_x,train_y)
    
        pred, confidence_level = model_fema_original.predict(test_x,4)
    
        cm_fema_original = confusion_matrix(test_y,pred)
        bal_acc_original = balanced_accuracy_score(test_y, pred)
        acc_original = accuracy_score(test_y, pred)
    
        #print the accuracy of original solution
        if DEBUG:
            print('**********',dataset_name,'**********',i)
            print('********** original **********')
            print(cm_fema_original) 
            print(bal_acc_original, acc_original)
    
    
        #Run the Semi Supervised FEMa
        model_semi = fema_semi.FEMaSemiSupervisedClassifier(k=50,basis=fema_semi.Basis.shepardBasis)
        model_semi.fit(train_x,train_y,uknw_x,2)
        
        pred, confidence_level = model_semi.predict(test_x,4)
        
        cm_fema_semi = confusion_matrix(test_y,pred)
        bal_acc_semi = balanced_accuracy_score(test_y, pred)
        acc_semi = accuracy_score(test_y, pred)
        
        #print the accuracy of original solution
        if DEBUG:
            print('********** semi **********')
            print(cm_fema_semi) 
            print(bal_acc_semi)
            print(acc_semi)
    
        return


In [4]:
for dataset in ml_datasets: 
    data = fetch_openml(dataset,version=1)
    X, y = data['data'], data['target']

    X = X.select_dtypes(include=np.number)

    le = preprocessing.LabelEncoder()
    le.fit(y)
    y = le.transform(y)

    if DEBUG:
        print(dataset, X.shape, y.shape, type(X), type(y))
        print(X.dtypes)
    
    experiment(dataset_name=dataset, data_x=X, data_y=y, test_size=0.90,n_runs=3,DEBUG=True)

********** credit-g **********
train_size 100 classes: {0, 1}
test_size 810 classes: {0, 1}
uknw_size 90 classes: {0, 1}
********** credit-g ********** 0
********** original **********
[[ 37 208]
 [ 72 493]]
0.5117933899223406 0.654320987654321
90 2
********** semi **********
[[ 21 224]
 [ 55 510]]
0.49418457648546144
0.6555555555555556
********** airlines **********
train_size 53938 classes: {0, 1}
test_size 436901 classes: {0, 1}
uknw_size 48544 classes: {0, 1}


/home/danillorp/Área de Trabalho/github/fema/src/fem_basis.py:33: RuntimeWarning: divide by zero encountered in divide
  dist = 1.0/(dist**z)
/home/danillorp/Área de Trabalho/github/fema/src/fem_basis.py:34: RuntimeWarning: invalid value encountered in divide
  weitghs = dist[mask]/sum(dist[mask]+0.0000000001)


KeyboardInterrupt: 